## Introduction to csv & sqlite with Python

<sup>The following was created with Jupyter Notebooks just to make it easier to add notes and directions - all code can be run under standard Python 3 (only uses built-in & standard libraries) just copy the code portions into your favorite python editor and save the books.csv and dvds.json files to the same directory as project.</sup>

<p>
    To load we will use the following libraries
    <ul><li><b>csv</b></li><li><b>sqlite3</b></li></ul>
</p>

<p>A csv file contains values seperated with a common character. Besides Json these are probably the most common way to save data exported from or to be imported into a database. It is also common with spreadsheets but <i>beware</i> as Excel or Sheets files are not automatically "csv"s. Common delimiters (what we call the character chosen to seperate values) include:</p>
    <ul>
        <li>comma (,)</li>
        <li>semicolon (;)</li>
        <li>pipe (|)</li>
        <li>whitespace or tabs</li>
    </ul>
    
<p>These files are so common that a library to handle them is standard with Python as is the library for communicating with SQLite. An example is below, note to run this you would just have to create a database called "Inventory"</p>

#### Basic Algorithm of Example 1
<p>First we connect to the database then set up a cursor and create the table (you don't have to create the table in code - doing this to show its possible and so it is easier to run - load the data using csv as a dictionary and build an insert query by looping over the dictionary with a list comprehension (covered in class). Then it is just connect and "executemany" - execute runs only a single command.</p>

In [4]:
import csv
import sqlite3

'''
  This code will load a a csv object (our books) then insert them into the database "Inventory"
  note - this is very much the simpliest version (which is great if data is great) but the basic
  idea doesn't change with complexity. It just adds validation, logging/error handling, and full functions
  for an API.
'''

con = sqlite3.connect("Inventory.db")
cur = con.cursor()

### create the table on the database (not IF NOT EXISTS is a constraint that does exactly as stated)
### note the 3 single quotes (''') is used for multiline strings as opposed to string concat ("foo " + "bar")
cur.execute('''CREATE TABLE IF NOT EXISTS book (
                isbn INTEGER PRIMARY KEY
                             UNIQUE
                             NOT NULL,
                title     TEXT,
                author    VARCHAR (40),
                genre     VARCHAR (40),
                tot_pages INTEGER,
                editor    VARCHAR (40));''')

with open('books.csv', 'r') as book_file:
    books_dict = csv.DictReader(book_file, delimiter=',') #default is comma; adding just to show parameter
    
    insert_vals =  [(k['isbn'], k['title'], k['author'], k['genre'], 
                     k['tot_pages'], k['editor']) 
                   for k in books_dict]
    
cur.executemany("INSERT INTO book VALUES (?,?,?,?,?,?);", insert_vals)
### the placeholders above (question marks) allow the library to escape everything preventing sql injection attacks
### Now if you weren't inserting all the values or the order wasn't the same - you would loop over the headers to build query
###        CSVs usually contain all the data in the same order as the table which is why they are so useful

In [5]:
### Need to actually print out the data returned from this (or assign to a data structure and analyze)
for i in cur.execute("select * from book;"):
    print(i)

(183947, "'the art of war'", "'sun tzu'", '129', "'matt markson'", None)
(201933, "'the way of kings'", "'brandon sanderson'", '420', "''", None)
(17384, "'elric of melnibone'", "'michael moorcock'", '181', "''", None)
(837211, "'the storyteller'", "'jodi picoult'", '178', "''", None)
(123, "'kinderund hausmarchen - grimms fairytales'", "'jakob and wilhelm grimm'", '585', "'patrick cohen'", None)
(927189, "'fairy tail - volume 1'", "'hiro mashima'", '367', "'kodansha'", None)


In [6]:
con.close() 
#close the connection (so the table/database isn't locked - see deadlock lecture)

## Using Json & a note on placeholders

We will move towards an API or Handler setup as we continue in the class, but we are starting with just a simple procedural program to go through the basic functions & procedures used.

#### Placeholders
Placeholders are used with SQL as a way of avoiding SQL Injection when sending user provided input to the database - with every non-assembly language I've worked with. Here we use it with an INSERT statement but it could be used with SELECT, UPDATE, or DELETE as well. Typically in the WHERE portion, such as <i>SELECT field1 FROM table1 <b>WHERE field2 = ?</b></i>. 

We will cover more about how these work in class, but for the example code just know the standard, unnamed, way to use a placeholder is with a "?" then reference by position (first example). If we instead needed named placeholders, for this class we will use the <i>format ":{field_name}"</i> such that the <b>sku</b> field would be <i>:sku</i>. Further, that <b><i>this does not replace the need to validate user input</i></b> but is used with it to provide security.

#### Json

* See first weeks lesson on Python & Json for any review needed


In [51]:
#load json into database
import json
# import sqlite3 #already imported - just uncomment if not using first part

'''
  This code will load a a json object (our dvds) then insert them into the database "Inventory"
  note - this is very much the simpliest version (which is great if data is great) but the basic
  idea doesn't change with complexity.
'''

con = sqlite3.connect("Inventory.db")
cur = con.cursor()

### create the table on the database (not IF NOT EXISTS is a constraint that does exactly as stated)
### note the 3 single quotes (''') is used for multiline strings
cur.execute('''CREATE TABLE IF NOT EXISTS dvds (
                    sku    INTEGER        PRIMARY KEY
                           UNIQUE
                           NOT NULL,
                    title  TEXT,
                    price  NUMERIC (8, 2),
                    genre  VARCHAR (40),
                    length INTEGER);''')

with open("dvds.json") as json_file:
    dvd_json = json.load(json_file)
    ### dvd_json is now a json object (basically a dictionary)

"""
The below can be optimized (and will be as we move further into functional programming & building handlers):
Currently it: 
1. Builds a string with place holders by creating a string of the field names (dict keys)
2. Then a list of keys (list comprehension) as placeholders (":{sku}, :{title}, etc")
3. Then joining our placeholders (same as 1)
4. Finally, using string's format method to build our query string (adding the strings from 1&3 to it)

"""

json_keys = dvd_json[0].keys()
field_names = ", ".join(json_keys)
placeholders = [":{}".format(k) for k in json_keys]
placeholders = ", ".join(placeholders)
query = "INSERT INTO dvds({}) VALUES ({});".format(field_names, placeholders)

# Finally just insert each one at a time (we will talk about batchs & transactions in a later lesson)
for rows in dvd_json:
    cur.execute(query, rows)

# And check the data
for i in cur.execute("select * from dvds;"):
    print(i)

INSERT INTO dvds(sku, title, price, genre, length) VALUES (:sku, :title, :price, :genre, :length);
{'sku': 8283, 'title': 'return of the jedi', 'price': 47.99, 'genre': 'science fiction', 'length': 90}
INSERT INTO dvds(sku, title, price, genre, length) VALUES (:sku, :title, :price, :genre, :length);
{'sku': 73891, 'title': 'the neverending story', 'price': 24.99, 'genre': 'fantasy', 'length': 120}
INSERT INTO dvds(sku, title, price, genre, length) VALUES (:sku, :title, :price, :genre, :length);
{'sku': 27811, 'title': 'the princess bride', 'price': 14.99, 'genre': 'fantasy', 'length': 90}
INSERT INTO dvds(sku, title, price, genre, length) VALUES (:sku, :title, :price, :genre, :length);
{'sku': 162543, 'title': 'how i met your mother - complete series', 'price': 94.99, 'genre': 'comedy', 'length': 1222}
(8283, 'return of the jedi', 47.99, 'science fiction', 90)
(27811, 'the princess bride', 14.99, 'fantasy', 90)
(73891, 'the neverending story', 24.99, 'fantasy', 120)
(162543, 'how i met

In [52]:
con.close() #close the connection (so the table/database isn't locked)